In [1]:
import torch
import torch.nn as nn

In [175]:
class block(nn.Module):
    def __init__(
        self, in_channels, intermediate_channels, identity_downsample=None, stride=1
    ):
        super(block, self).__init__()
        self.expansion = 2
        self.conv1 = nn.Conv1d(
            in_channels, intermediate_channels, kernel_size=1, stride=1, padding=0, bias=False
        )
        self.bn1 = nn.BatchNorm1d(intermediate_channels)
        self.conv2 = nn.Conv1d(
            intermediate_channels,
            intermediate_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False
        )
        self.bn2 = nn.BatchNorm1d(intermediate_channels)
        self.conv3 = nn.Conv1d(
            intermediate_channels,
            intermediate_channels * self.expansion,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False
        )
        self.bn3 = nn.BatchNorm1d(intermediate_channels * self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x):
        identity = x.clone()
        x = self.conv1(x)
        print("Block", identity.shape)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)

        print("Block", x.shape, identity.shape)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)
            print("Block", identity.shape)

        x += identity
        x = self.relu(x)
        
        return x

In [187]:
b1 = block(1, 2, identity_downsample=None, stride=1)
b2 = block(4, 2, identity_downsample=None, stride=1)
b3 = block(4, 2, identity_downsample=None, stride=1)
b4 = block(4, 2, identity_downsample=None, stride=1)

identity_downsample = nn.Sequential(
            nn.Conv1d(
                16,
                32,
                kernel_size=1,
                stride=1,
                bias=False
            ),
            nn.BatchNorm1d(32),
)

b5 = block(16,16, identity_downsample)

In [200]:
X = torch.randn((20,1,300))

In [186]:
W = torch.randn((20,32,320))
identity_downsample = nn.Sequential(
    nn.Conv1d(
        32,
        16,
        kernel_size=1,
        stride=1,
        bias=False
    ),
    nn.BatchNorm1d(16),
)
m = nn.Conv1d(
        32,
        16,
        kernel_size=1,
        stride=1,
        bias=False
    )
identity_downsample(W).shape

torch.Size([20, 16, 320])

In [188]:
Z = torch.randn((20,16,32*10))
b5(Z).shape

Block torch.Size([20, 16, 320])
Block torch.Size([20, 32, 320]) torch.Size([20, 16, 320])
Block torch.Size([20, 32, 320])


torch.Size([20, 32, 320])

In [51]:
x = b1(X)
print(x.shape)
x = b2(x)
print(x.shape)
# x = b3(x)
# print(x.shape)
# x = b4(x)
# print(x.shape)
x = torch.flatten(x,1)
print(x.shape)


torch.Size([20, 4, 300])
torch.Size([20, 4, 300])
torch.Size([20, 1200])


In [198]:
class ResNet(nn.Module):
    
    def __init__(self, num_classes=3):
        super(ResNet, self).__init__()
        self.in_channels = 1
        self.conv1 = nn.Conv1d(1, 16, kernel_size=5, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm1d(16)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)

        # Essentially the entire ResNet architecture are in these 4 lines below
        self.layer1 = nn.Sequential(block(16,8), block(16, 8))

        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(16, num_classes)

    def forward(self, x):
        print(x.shape)
        x = self.conv1(x)
        print(x.shape)
        x = self.bn1(x)
        print(x.shape)
        x = self.relu(x)
        x = self.maxpool(x)
        print(x.shape)
        x = self.layer1(x)
        print(x.shape)
        x = self.avgpool(x)
        print(x.shape)
        x = torch.flatten(x, 1)
        print(x.shape)
        x = self.fc(x)
        print(x.shape)
        return x



In [199]:
r = ResNet(num_classes=10)

In [201]:
r(X).shape

torch.Size([20, 1, 300])
torch.Size([20, 16, 151])
torch.Size([20, 16, 151])
torch.Size([20, 16, 76])
Block torch.Size([20, 16, 76])
Block torch.Size([20, 16, 76]) torch.Size([20, 16, 76])
Block torch.Size([20, 16, 76])
Block torch.Size([20, 16, 76]) torch.Size([20, 16, 76])
torch.Size([20, 16, 76])
torch.Size([20, 16, 1])
torch.Size([20, 16])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (20x16 and 32x10)